### Gerenciando Estado no LangGraph
____

O gerenciamento de estado é essencial em fluxos de trabalho com LangGraph, onde um StateGraph rastreia entradas, saídas e dados intermediários.
Entender como definir e gerenciar o estado corretamente melhora a confiabilidade e o controle em workflows baseados em IA.

#### **Máquinas de Estado e LangGraph**

Sistemas baseados em estado não são novidade.
Máquinas de Estado — como controladores de semáforo — mudam entre estados pré-definidos com base em lógica e condições.

No LangGraph, os fluxos seguem a mesma ideia:

* Nodes (nós) representam estados. - é uma função ou objeto `Runnable` que
representa uma unidade de trabalho.

* Edges (arestas) definem as transições entre eles.

📌 Exemplo simples de máquina de estados (semáforo):

In [ ]:
traffic_light = {"green": "yellow", "yellow": "red", "red": "green"}

state = "green"
for _ in range(4):
    print(f"Current: {state}")
    state = traffic_light[state]

Current: green
Current: yellow
Current: red
Current: green


#### Definindo Schemas de Estado:
___

Schemas de estado definem quais dados são armazenados e atualizados à medida que o fluxo de trabalho roda.

Duas abordagens comuns:

1. TypedDict

2. Pydantic

In [ ]:
from typing import TypedDict


class State(TypedDict):
    """TypedDict representing the state with user_string and action_type.

    Args:
        user_string (str): The input string from the user.
        action_type (str): The type of action to perform on the user_string.

    """

    user_string: str
    action_type: str


# Exemplo de uso
state: State = {"user_string": "hello", "action_type": "reverse"}
print(state)

{'user_string': 'hello', 'action_type': 'reverse'}


* Usando `Pydantic` * (mais robusto e com validação)

In [ ]:
from pydantic import BaseModel, ValidationError


class State(BaseModel):
    """A pydantic class representing the state with user_string and action_type.

    Args:
        user_string (str): The input string from the user.
        action_type (str): The type of action to perform on the user_string.

    """

    user_string: str
    action_type: str


try:
    state = State(user_string="hello", action_type="reverse")
    print(state.model_dump())
except ValidationError as e:
    print("Validation error:", e)

{'user_string': 'hello', 'action_type': 'reverse'}


**Resumo**:

* Use TypedDict → prototipagem rápida, pouca validação.

* Use Pydantic → ambientes de produção, garante consistência e validação dos dados.

#### **Considerações Finais**
___

Ao projetar `workflows` com `LangGraph`, é **fundamental dedicar tempo para definir um estado bem estruturado**, *alinhado às necessidades da sua aplicação*.
O `estado` não é apenas um `“lugar para guardar variáveis”`, mas sim o ``núcleo do controle do fluxo``, garantindo que cada transição entre nós siga de forma previsível.

##### Escolha da abordagem para gerenciar estado:

🔹 TypedDict → quando usar?

  * Ideal para casos simples e rápidos, onde você precisa apenas de uma estrutura clara de chave → valor.
  * Serve bem em prototipagem, quando você está testando fluxos e não precisa de validação rigorosa.
  * Exemplo: guardar apenas o texto do usuário e uma flag com a ação que será tomada.

In [ ]:
from typing import TypedDict


class SimpleState(TypedDict):
    """TypedDict representing the state with user_string and action_type.

    Args:
        user_string (str): The input string from the user.
        action_type (str): The type of action to perform on the user_string.

    """

    user_message: str
    action: str


state: SimpleState = {"user_message": "Oi LangGraph!", "action": "responder"}
print(state)

{'user_message': 'Oi LangGraph!', 'action': 'responder'}


🔹 Pydantic → quando usar?

* Recomendado para fluxos complexos em produção, onde você precisa de validação automática dos dados.
* Ajuda a evitar inconsistências (ex.: usuário envia um número onde você esperava texto).
* Permite adicionar regras de negócio, tipos mais sofisticados e até validações customizadas.

In [ ]:
from pydantic import BaseModel, Field, ValidationError


class ValidatedState(BaseModel):
    """Pydantic model representing the validated state with user_message and action.

    Attributes:
        user_message (str): Mensagem enviada pelo usuário.
        action (str): Tipo de ação a ser tomada.

    """

    user_message: str = Field(
        ..., min_length=1, description="Mensagem enviada pelo usuário"
    )
    action: str


# Estado válido
state = ValidatedState(user_message="Oi LangGraph!", action="responder")
print(state.model_dump())

# Estado inválido → dispara erro de validação
try:
    invalid_state = ValidatedState(user_message="", action="responder")
except ValidationError as e:
    print("Erro de validação:", e)

{'user_message': 'Oi LangGraph!', 'action': 'responder'}
Erro de validação: 1 validation error for ValidatedState
user_message
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short


`O Problema`: **Por padrão, quando um nó retorna um valor para um campo do estado (como uma lista de mensagens), ele sobrescreve o valor anterior.**

Excelente pergunta 🚀 — esse é um dos conceitos mais poderosos (e que às vezes passa despercebido) quando a gente começa a usar **LangGraph**: o uso de **`Annotated` com redutores**.
Vou explicar como instrutor de **LangGraph / LangChain / Python** e depois te dar **2 exemplos práticos** para você testar.

---

## 📌 O Problema

Por padrão, em LangGraph, quando um nó atualiza um campo do estado, ele **substitui** o valor antigo.

Exemplo:

* Estado inicial: `{"messages": ["Oi"]}`
* Nó X retorna: `{"messages": ["Olá"]}`
* Resultado: `{"messages": ["Olá"]}`
  👉 A mensagem `"Oi"` foi perdida!

---

## 📌 A Solução: `Annotated` + redutores

O **truque** é usar `typing.Annotated` com uma função redutora (como `operator.add`).
Assim, ao invés de substituir, o LangGraph **combina o valor anterior com o novo**.

* `Annotated[List[str], add]` → concatena listas
* `Annotated[int, add]` → soma números

Isso cria **estados acumulativos**, onde cada nó pode contribuir sem apagar o que já existia.

---

## 🔹 Exemplo 1: Acumular mensagens em uma lista

```python
from typing import List, Annotated
from operator import add

# Definindo o schema do estado
class State(TypedDict):
    messages: Annotated[List[str], add]

# Estado inicial
state: State = {"messages": ["Oi"]}

# Nó 1 adiciona uma nova mensagem
node1_output = {"messages": ["Tudo bem?"]}
state["messages"] = add(state["messages"], node1_output["messages"])

# Nó 2 adiciona outra mensagem
node2_output = {"messages": ["Sim, e você?"]}
state["messages"] = add(state["messages"], node2_output["messages"])

print(state)
```

**Saída esperada:**

```python
{'messages': ['Oi', 'Tudo bem?', 'Sim, e você?']}
```

👉 Cada nó contribuiu, e o estado **preservou o histórico de mensagens**.

---

## 🔹 Exemplo 2: Acumular contagem de eventos

```python
from typing import Annotated
from operator import add

class State(TypedDict):
    counter: Annotated[int, add]

# Estado inicial
state: State = {"counter": 0}

# Nó green_light incrementa
node1_output = {"counter": 1}
state["counter"] = add(state["counter"], node1_output["counter"])

# Nó yellow_light incrementa
node2_output = {"counter": 1}
state["counter"] = add(state["counter"], node2_output["counter"])

# Nó red_light incrementa
node3_output = {"counter": 1}
state["counter"] = add(state["counter"], node3_output["counter"])

print(state)
```

**Saída esperada:**

```python
{'counter': 3}
```

👉 Cada nó incrementa o contador sem sobrescrever o valor anterior.

---

## 💡 Por que isso é genial?

* Os **nós colaboram** na construção do estado.
* Você pode guardar **histórico de mensagens**, **logs de execução** ou **estatísticas de uso** sem esforço extra.
* É **declarativo e simples** → só anotar o tipo e pronto!